# Baseline 모델 구축

## 목표
- Macro-F1 Score > 0.9 달성
- 다양한 모델 실험 및 최적 모델 선정

## 전략
1. 간단한 전처리 + 기본 모델로 시작
2. 점진적으로 복잡한 모델 적용
3. 교차 검증으로 안정성 확인

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import warnings
warnings.filterwarnings('ignore')

from sklearn.model_selection import train_test_split, StratifiedKFold, cross_val_score
from sklearn.preprocessing import StandardScaler, LabelEncoder
from sklearn.metrics import f1_score, classification_report, confusion_matrix, make_scorer

# Models
from sklearn.ensemble import RandomForestClassifier, ExtraTreesClassifier
from sklearn.linear_model import LogisticRegression
from xgboost import XGBClassifier
from lightgbm import LGBMClassifier
from catboost import CatBoostClassifier

import time
from tqdm import tqdm

# 재현성을 위한 시드 설정
RANDOM_STATE = 42
np.random.seed(RANDOM_STATE)

print('라이브러리 로드 완료')

## 1. 데이터 로드 및 전처리

In [ ]:
# 데이터 로드
train = pd.read_csv('../data/train.csv')
test = pd.read_csv('../data/test.csv')
submission = pd.read_csv('../data/sample_submission.csv')

# 피처와 타겟 분리
feature_cols = [col for col in train.columns if col not in ['ID', 'target']]
X_train_full = train[feature_cols]
y_train_full = train['target']
X_test = test[feature_cols]

print(f"Train shape: {X_train_full.shape}")
print(f"Test shape: {X_test.shape}")
print(f"Target classes: {sorted(y_train_full.unique())}")
print(f"Number of classes: {y_train_full.nunique()}")

In [ ]:
# Train/Validation 분할 (홀드아웃 검증용)
X_train, X_val, y_train, y_val = train_test_split(
    X_train_full, y_train_full, 
    test_size=0.2, 
    random_state=RANDOM_STATE, 
    stratify=y_train_full
)

print(f"Train set: {X_train.shape}")
print(f"Validation set: {X_val.shape}")
print(f"\nTarget distribution in train:")
print(y_train.value_counts(normalize=True).sort_index())

In [ ]:
# 스케일링
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_val_scaled = scaler.transform(X_val)
X_train_full_scaled = scaler.fit_transform(X_train_full)
X_test_scaled = scaler.transform(X_test)

print("스케일링 완료")

## 2. Baseline 모델 학습 및 평가

In [ ]:
# 평가 함수
def evaluate_model(model, X_train, y_train, X_val, y_val, model_name):
    """모델 학습 및 평가"""
    start_time = time.time()
    
    # 학습
    model.fit(X_train, y_train)
    
    # 예측
    y_train_pred = model.predict(X_train)
    y_val_pred = model.predict(X_val)
    
    # Macro F1 Score 계산
    train_f1 = f1_score(y_train, y_train_pred, average='macro')
    val_f1 = f1_score(y_val, y_val_pred, average='macro')
    
    # 각 클래스별 F1 Score
    class_f1 = f1_score(y_val, y_val_pred, average=None)
    
    elapsed_time = time.time() - start_time
    
    print(f"\n{'='*50}")
    print(f"Model: {model_name}")
    print(f"{'='*50}")
    print(f"Train Macro F1: {train_f1:.4f}")
    print(f"Val Macro F1: {val_f1:.4f}")
    print(f"Training time: {elapsed_time:.2f} seconds")
    print(f"\nClass-wise F1 scores:")
    for i, f1 in enumerate(class_f1):
        print(f"  Class {i}: {f1:.4f}")
    
    return model, val_f1, class_f1

In [ ]:
# 모델 정의
models = {
    'RandomForest': RandomForestClassifier(
        n_estimators=200,
        max_depth=15,
        min_samples_split=5,
        min_samples_leaf=2,
        random_state=RANDOM_STATE,
        n_jobs=-1,
        class_weight='balanced'
    ),
    'ExtraTrees': ExtraTreesClassifier(
        n_estimators=200,
        max_depth=15,
        min_samples_split=5,
        min_samples_leaf=2,
        random_state=RANDOM_STATE,
        n_jobs=-1,
        class_weight='balanced'
    ),
    'XGBoost': XGBClassifier(
        n_estimators=300,
        max_depth=8,
        learning_rate=0.1,
        subsample=0.8,
        colsample_bytree=0.8,
        random_state=RANDOM_STATE,
        n_jobs=-1,
        objective='multi:softprob',
        eval_metric='mlogloss'
    ),
    'LightGBM': LGBMClassifier(
        n_estimators=300,
        max_depth=8,
        learning_rate=0.1,
        num_leaves=31,
        subsample=0.8,
        colsample_bytree=0.8,
        random_state=RANDOM_STATE,
        n_jobs=-1,
        objective='multiclass',
        metric='multi_logloss',
        class_weight='balanced',
        verbose=-1
    ),
    'CatBoost': CatBoostClassifier(
        iterations=500,
        depth=8,
        learning_rate=0.1,
        random_state=RANDOM_STATE,
        verbose=False,
        auto_class_weights='Balanced'
    )
}

# 결과 저장
results = {}

In [ ]:
# 모든 모델 학습 및 평가
for name, model in models.items():
    trained_model, val_f1, class_f1 = evaluate_model(
        model, X_train_scaled, y_train, X_val_scaled, y_val, name
    )
    results[name] = {
        'model': trained_model,
        'val_f1': val_f1,
        'class_f1': class_f1
    }

In [ ]:
# 결과 요약
print("\n" + "="*60)
print("모델 성능 요약 (Validation Set)")
print("="*60)

summary_df = pd.DataFrame([
    {'Model': name, 'Macro F1': res['val_f1']} 
    for name, res in results.items()
]).sort_values('Macro F1', ascending=False)

display(summary_df)

best_model_name = summary_df.iloc[0]['Model']
best_f1 = summary_df.iloc[0]['Macro F1']

print(f"\n🏆 Best Model: {best_model_name}")
print(f"   Macro F1 Score: {best_f1:.4f}")

if best_f1 >= 0.9:
    print("\n✅ 목표 달성! (Macro F1 >= 0.9)")
else:
    print(f"\n⚠️ 목표 미달 (현재: {best_f1:.4f}, 목표: 0.9)")
    print("   추가 개선이 필요합니다.")

## 3. K-Fold 교차 검증

In [ ]:
# 베스트 모델로 K-Fold 교차 검증
best_model = results[best_model_name]['model']

# Stratified K-Fold
skf = StratifiedKFold(n_splits=5, shuffle=True, random_state=RANDOM_STATE)

# Macro F1 scorer
macro_f1_scorer = make_scorer(f1_score, average='macro')

# 교차 검증
print(f"\n{best_model_name} 모델 5-Fold 교차 검증 중...")
cv_scores = cross_val_score(
    best_model.__class__(**best_model.get_params()),
    X_train_full_scaled,
    y_train_full,
    cv=skf,
    scoring=macro_f1_scorer,
    n_jobs=-1
)

print(f"\n5-Fold CV Results:")
print(f"Scores: {cv_scores}")
print(f"Mean: {cv_scores.mean():.4f}")
print(f"Std: {cv_scores.std():.4f}")

## 4. Feature Importance 분석

In [ ]:
# Feature Importance (Tree 기반 모델만)
if hasattr(best_model, 'feature_importances_'):
    feature_importance = pd.DataFrame({
        'feature': feature_cols,
        'importance': best_model.feature_importances_
    }).sort_values('importance', ascending=False)
    
    print(f"\nTop 15 Important Features ({best_model_name}):")
    display(feature_importance.head(15))
    
    # 시각화
    plt.figure(figsize=(10, 8))
    top_features = feature_importance.head(20)
    plt.barh(range(len(top_features)), top_features['importance'].values)
    plt.yticks(range(len(top_features)), top_features['feature'].values)
    plt.xlabel('Importance')
    plt.title(f'Top 20 Feature Importances - {best_model_name}')
    plt.tight_layout()
    plt.show()

## 5. 최종 모델 학습 및 예측

In [ ]:
# 전체 데이터로 최종 모델 학습
print(f"\n최종 모델 학습 중 ({best_model_name})...")

final_model = best_model.__class__(**best_model.get_params())
final_model.fit(X_train_full_scaled, y_train_full)

# 테스트 데이터 예측
test_predictions = final_model.predict(X_test_scaled)

print("예측 완료!")
print(f"\n예측 분포:")
print(pd.Series(test_predictions).value_counts().sort_index())

In [ ]:
# 제출 파일 생성
submission['target'] = test_predictions

# 저장
submission_filename = f'../submissions/baseline_{best_model_name.lower()}_f1_{best_f1:.4f}.csv'
submission.to_csv(submission_filename, index=False)

print(f"\n✅ 제출 파일 생성 완료: {submission_filename}")
print("\n제출 파일 미리보기:")
display(submission.head(10))

## 6. 개선 방향

In [ ]:
print("="*60)
print("📈 성능 개선 방향")
print("="*60)

if best_f1 < 0.9:
    print("""
현재 성능이 목표에 미달하므로 다음 방법들을 시도해야 합니다:

1. Feature Engineering 강화
   - 피처 간 상호작용 (곱셈, 나눗셈)
   - 통계적 특징 (그룹별 평균, 표준편차)
   - 다항식 특징
   - PCA, t-SNE 등 차원 축소

2. 하이퍼파라미터 최적화
   - Optuna를 이용한 베이지안 최적화
   - GridSearchCV 또는 RandomizedSearchCV

3. 앙상블 방법
   - Voting Classifier
   - Stacking
   - Blending

4. 클래스 불균형 처리
   - SMOTE 등 오버샘플링
   - 클래스 가중치 조정

5. 신경망 모델
   - TabNet
   - Deep Neural Network
    """)
else:
    print("""
✅ 목표 성능을 달성했습니다!

추가 개선을 위한 제안:
1. 앙상블로 안정성 향상
2. 후처리(Post-processing) 적용
3. Pseudo-labeling 고려
    """)

print(f"\n현재 최고 성능: {best_f1:.4f} ({best_model_name})")